In [1]:
import json
from web3 import Web3, HTTPProvider
import ipfs_api
from keras.models import load_model

2023-10-29 07:42:43.834274: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-29 07:42:43.884313: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-29 07:42:43.885419: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-29 07:42:45.113071: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
print(ipfs_api.my_id())

12D3KooWRYxPKAzZdRmFR154HEFGjT4KrxaUWmQ7q9sAXWNNuVfa


In [3]:
blockchain_address = 'http://127.0.0.1:9545'
web3 = Web3(HTTPProvider(blockchain_address))
web3.is_connected()

True

In [4]:
web3.eth.defaultAccount = web3.eth.accounts[0]
print(web3.eth.accounts[0])

0xEa9fdd0659F5a69ad248D2F7fCCe343449E72636


In [5]:
compiled_contract_path = '/home/administrator/docker-eth/build/contracts/ids.json'

with open(compiled_contract_path) as file:
    contract_json = json.load(file) 
    contract_abi = contract_json['abi']

In [6]:
deployed_contract_address = '0x9B211180B1E9189305897AfD2f1f82bf9815FCf5'

contract = web3.eth.contract(
    address = deployed_contract_address, abi = contract_abi)

In [7]:
import time
import psutil

In [8]:
def generate_transactions(web3, contract, accounts, iterations):
    p = psutil.Process(185078) #truffle
    q = psutil.Process(133348) #IPFS
    cpu_usages_p = []
    ram_usages_p = []
    cpu_usages_q = []
    ram_usages_q = []
    
    for _ in range(iterations):
        cpu_percent_p_before = p.cpu_percent()
        cpu_percent_q_before = q.cpu_percent()
        for i in range(10):
            cid = ipfs_api.publish("./model_federated.h5")
            tx_hash = contract.functions.sendHash(cid).transact({'from': web3.eth.accounts[0]})
            tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
            print(f'Sent transaction {i+1}/{iterations*10} with hash: {tx_hash.hex()}')
        cpu_percent_p_after = p.cpu_percent()
        cpu_percent_q_after = q.cpu_percent()
        cpu_usage_p = cpu_percent_p_after - cpu_percent_p_before
        cpu_usage_q = cpu_percent_q_after - cpu_percent_q_before
        cpu_usages_p.append(cpu_usage_p)
        cpu_usages_q.append(cpu_usage_q)
        
        mem_info_p = p.memory_info()
        ram_usage_p = mem_info_p.rss / (1024 * 1024)
        ram_usages_p.append(ram_usage_p)
        
        mem_info_q = q.memory_info()
        ram_usage_q = mem_info_q.rss / (1024 * 1024)
        ram_usages_q.append(ram_usage_q)
    
    average_cpu_usage_p = sum(cpu_usages_p) / len(cpu_usages_p)
    print(f'Average CPU Truffle: {average_cpu_usage_p} %')
    average_ram_usage_p = sum(ram_usages_p) / len(ram_usages_p)
    print(f'Average RAM Truffle: {average_ram_usage_p} MB')
    cpu_usages_p.clear()
    ram_usages_p.clear()
    
    average_cpu_usage_q = sum(cpu_usages_q) / len(cpu_usages_q)
    print(f'Average CPU IPFS: {average_cpu_usage_q} %')
    average_ram_usage_q = sum(ram_usages_q) / len(ram_usages_q)
    print(f'Average RAM IPFS: {average_ram_usage_q} MB')
    cpu_usages_q.clear()
    ram_usages_q.clear()

In [9]:
start_time1 = time.time()

generate_transactions(web3, contract, web3.eth.defaultAccount, 10) # send 1000 transactions

end_time1 = time.time()

latency1 = end_time1 - start_time1
tps1 = 1000 / int(latency1)
print(f'Total transactions sent: 1000')
print(f'Latency: {int(latency1)}')
print(f'Transaction per second (tps): {tps1}')

Sent transaction 1/100 with hash: 0xd57941f88e53791512433bc2ace60c61eeb5abf4f34a2e0420b0dcff40b7122a
Sent transaction 2/100 with hash: 0x8ceccf560f162ccd273d6c263dfc1f6fa8c4b73370a96f9f3fc88e1a7a2ffca0
Sent transaction 3/100 with hash: 0x12ab30b727160a3e29973b5cb7337e27f6c205fa156b6b0e99b65927afd8a631
Sent transaction 4/100 with hash: 0xa85bf198f0ee7556ecc67700a735c9e098ea000fee43ca617896ae67d6b1a704
Sent transaction 5/100 with hash: 0x10611bf3fefad143e6f337acb51d041f82c325ead22fbce10a92647928152e05
Sent transaction 6/100 with hash: 0xcc2b540be6f123ab60872c5c83989cf5c2f343f5e6924ebf177f54aa2b8a2d19
Sent transaction 7/100 with hash: 0xa8f654fc4aa9f935ab073f6dbcd46f1bfb730ed6f012ba16fd6cc582f47462ca
Sent transaction 8/100 with hash: 0xf5769103954bcb688e0743b15b44fed094df0e36283c449cb3e0a719c7cbc1c1
Sent transaction 9/100 with hash: 0x60b1af5d516b29d223af00e7a7d8732541ade5eed5980c2ac23f7a01ce4da79e
Sent transaction 10/100 with hash: 0xff4ea974e1d1c06737950ddfb5400285eada85bae51aa7384c22f9

In [10]:
start_time2 = time.time()

generate_transactions(web3, contract, web3.eth.defaultAccount, 20) # send 2000 transactions

end_time2 = time.time()

latency2 = end_time2 - start_time2
tps2 = 2000 / int(latency2)
print(f'Total transactions sent: 2000')
print(f'Latency: {int(latency2)}')
print(f'Transaction per second (tps): {tps2}')

Sent transaction 1/200 with hash: 0xd3d28b240f3e2d61cafbf5f2d97010bd390d974a26c7b01537fdbb71edae4d52
Sent transaction 2/200 with hash: 0x93a0614d5581d1ee651a1d3abad1841775b80cbca17efeedde106aba10589a7b
Sent transaction 3/200 with hash: 0xaa07a94edfbcf005b629d6d4076563cf950cdf2e1d44037647467b09e131bc4c
Sent transaction 4/200 with hash: 0xcf42eee2d673cd5f76d9703797194d9cdc2cec3e7f13289e91c7dda2e6e70a0e
Sent transaction 5/200 with hash: 0x91ea519de74c5e26c78b7124e78707f73f7b3ef2c539e5f93bc82f79fb44808c
Sent transaction 6/200 with hash: 0xa2b760b6437f0c6332cd1557c1feada6c56fb2e9205e98eda890bc5441fd07ec
Sent transaction 7/200 with hash: 0x1f493c3f6a0346a94bd9fd388059e98d1a96a6ec73a4149e70487e1bcfd33a87
Sent transaction 8/200 with hash: 0x779a25bd466821c3732c77eb1272a7a8ca30f4842fa5e99451f473c8e62dd9aa
Sent transaction 9/200 with hash: 0x470f7e51257293db5ad187a72573ed5f5a31d2174fed96e1a904c48307b0138d
Sent transaction 10/200 with hash: 0xc2d960e65d8659738a83ab760fb0864df4feba036f157da0e63c7e

In [11]:
start_time3 = time.time()

generate_transactions(web3, contract, web3.eth.defaultAccount, 30) # send 2000 transactions

end_time3 = time.time()

latency3 = end_time3 - start_time3
tps3 = 3000 / int(latency3)
print(f'Total transactions sent: 3000')
print(f'Latency: {int(latency3)}')
print(f'Transaction per second (tps): {tps3}')

Sent transaction 1/300 with hash: 0xab2c55f6dbf50edb05d8da1a4d8db2684632b2a47ebe4d9ee32efc3e03e206b5
Sent transaction 2/300 with hash: 0x973e8ac7d6d57422405c00edcd8424f9e9bec11846feec553dc682cc759da5c6
Sent transaction 3/300 with hash: 0xf0d28574a7bbbe661df1bad93ad2333c68176176236c351cb52171eb1d628c7f
Sent transaction 4/300 with hash: 0xaf2dfaa1d65dd9dae4ae6a083950cb53a653770a707698465ca3902b4a11b0ec
Sent transaction 5/300 with hash: 0x6bc58debe5706375a33ac9dd6f67830b7458ec69e400465db806ddb3645a3a67
Sent transaction 6/300 with hash: 0xa74b2975163e075b9de8e5ee69f6e623cfb5facbbff72c6b4126222eeced241e
Sent transaction 7/300 with hash: 0x2327169427f4a4a83302a506804daee8216064cdd4d1e65ffbf4b383b6eede50
Sent transaction 8/300 with hash: 0x44b9b75db3e17d45d78ec90571d9a79b71d65695c7d237a868c2b35519779c2b
Sent transaction 9/300 with hash: 0xe329f796febc49e749a35cedec4c8e6100c4479f4958cc481c16e666d14e6ec6
Sent transaction 10/300 with hash: 0x45002b7373c6c9e0cec461e97cf5eba87428efcbc0b535a313731e

In [12]:
start_time4 = time.time()

generate_transactions(web3, contract, web3.eth.defaultAccount, 40) # send 2000 transactions

end_time4 = time.time()

latency4 = end_time4 - start_time4
tps4 = 4000 / int(latency4)
print(f'Total transactions sent: 4000')
print(f'Latency: {int(latency4)}')
print(f'Transaction per second (tps): {tps4}')

Sent transaction 1/400 with hash: 0xbf7d6f13bc5e32e5babd17b91380b84342e8ea041ccf2ad202bab7ee0a3e657c
Sent transaction 2/400 with hash: 0xce48af7ea31b25cfc5759187d496fa230437fc7f6fbaed4de61e80dc25bf1614
Sent transaction 3/400 with hash: 0xc8c8ddbbe2892040462788c055f6f431d9e4fb24defb58cd84d190293aa59b60
Sent transaction 4/400 with hash: 0x9178592497143d5da0cc54e202f351fe8fa2dc62587f94c536aa3e57938c9c70
Sent transaction 5/400 with hash: 0x51cb10ce539870814e8a529056ce138aaa3ace4b779622a93da9738dd7286f6d
Sent transaction 6/400 with hash: 0x5921145691e6ebcf6e9f46e01707b836eb5b56b2abd22ef7091f09352e9076cd
Sent transaction 7/400 with hash: 0x45538efbb196f4f9f2f78958d46409fa546afef5f26cedc1c6e5a3d5fb82bf87
Sent transaction 8/400 with hash: 0x7d76c814cf0bf93ab8ac2e69a7a8ccd20219e136b258c6c51c5514c4ed6f3d46
Sent transaction 9/400 with hash: 0x0e6fc7b1047b1b0adb9aef4cad03019d89f69aaab3e7dcab6f09bcf3cf498ef2
Sent transaction 10/400 with hash: 0x5ed2c5d5ec97db4988bb8bafc346526a8f9a685ab54d9ba5db21d5

In [13]:
start_time5 = time.time()

generate_transactions(web3, contract, web3.eth.defaultAccount, 50) # send 5000 transactions

end_time5 = time.time()

latency5 = end_time5 - start_time5
tps5 = 5000 / int(latency5)
print(f'Total transactions sent: 5000')
print(f'Latency: {int(latency5)}')
print(f'Transaction per second (tps): {tps5}')

Sent transaction 1/500 with hash: 0xf6551edbe61f9b8cdb4c840ff9ba48f73b7d66f60611948bc099a39511cc4674
Sent transaction 2/500 with hash: 0xe223c2584c9ceb66eda68bc62cadc7d0a1bffebca9f8170b61d6b2194adbcca5
Sent transaction 3/500 with hash: 0xabf7227dc750d8b7949536e22544e85651f94addb68946b39d37b43136b29e63
Sent transaction 4/500 with hash: 0xa856673714b997333179f0aec0e9abd99cb957b0370681d24320851e22e008a1
Sent transaction 5/500 with hash: 0xebc18681c523d2505c07f20130cef445038130c96897a125c4f983092c848f2f
Sent transaction 6/500 with hash: 0xd12f9ecdc972bafbf89f0274928600677f1e12b5547981a9d4353879725b729e
Sent transaction 7/500 with hash: 0x52cfeb29f9cc4405494a1f4ed90bed385c8d032c7afaa5a988c1bd38f2595b00
Sent transaction 8/500 with hash: 0x2bc6ef01cfaee12aff869392d0f8c98b148d26969eabf380582316cca0236843
Sent transaction 9/500 with hash: 0xbbe0e02a179055fc4b2b5b44df4421e3f7e6704f6d70ca03e42975a34260b16e
Sent transaction 10/500 with hash: 0xc3d7d13c4843767eb570a267fc90efa232240cc915c4c9c855c1cf

In [14]:
start_time6 = time.time()

generate_transactions(web3, contract, web3.eth.defaultAccount, 60) # send 2000 transactions

end_time6 = time.time()

latency6 = end_time6 - start_time6
tps6 = 6000 / int(latency6)
print(f'Total transactions sent: 6000')
print(f'Latency: {int(latency6)}')
print(f'Transaction per second (tps): {tps6}')

Sent transaction 1/600 with hash: 0xff32e740e908af87c35f46272061ba3b0b3e628b5b73e2a437f50606fef1c9e0
Sent transaction 2/600 with hash: 0xcd2c192977016c1671e7d57869de32294eee373c6f4c2e812627dc519b7c2ac3
Sent transaction 3/600 with hash: 0xc40595833d4f56b73d66e53e9104a8416f5a798917250e1098fc63f314d6e001
Sent transaction 4/600 with hash: 0x04c1f24e035121887e7157a9e8e9371d7ab843e8fae8bbac6587974cdf690176
Sent transaction 5/600 with hash: 0x0817b95762995876a77a63abda1be82e35c36f10b0849fc4f89a5a3577a99b28
Sent transaction 6/600 with hash: 0xe70e5bad60b693e35afbe602bdf1876783c07149c40c26167ffdbbdb3734d459
Sent transaction 7/600 with hash: 0x8505ef04d1583e09eb2b50560c253f8d4ea7d70ca12fa354df5ad8c63ffbe5db
Sent transaction 8/600 with hash: 0xa3153fc2582d41135d06af17a1a97dc20050e98b45c32eaba6253a0551768401
Sent transaction 9/600 with hash: 0x0df23a7aa2e186c95155706d7b3a8fdfebafea7e42acb5d9871c5e53f008faae
Sent transaction 10/600 with hash: 0x5c8e5c49cd7a261506819211f6f93cb92bde26eb35ef82432ab9b9

In [15]:
start_time7 = time.time()

generate_transactions(web3, contract, web3.eth.defaultAccount, 70) # send 2000 transactions

end_time7 = time.time()

latency7 = end_time7 - start_time7
tps7 = 7000 / int(latency7)
print(f'Total transactions sent: 7000')
print(f'Latency: {int(latency7)}')
print(f'Transaction per second (tps): {tps7}')

Sent transaction 1/700 with hash: 0x7ff7701f61f4ecbd2349af7516080ad107be678aba8da4021a1e30082c630a30
Sent transaction 2/700 with hash: 0x882b53c301580d0d3722505aec4ec059a5fc909d977c699961ad37c4570f9abd
Sent transaction 3/700 with hash: 0x50e6b8dd5d75a1680035408f15854eb3df4057bac7ddfb4d94732a8c7d401d33
Sent transaction 4/700 with hash: 0x04e118942f9c321d049b0ba791a045fff68b12471c5dfac8e47148b0e3ad7679
Sent transaction 5/700 with hash: 0xd94ca472c18d478c5e04bfa784e3c1b6721bd572bdec0de1901acbf9754e2b3d
Sent transaction 6/700 with hash: 0xc28f106537817b67f495abcae0301a3789f3e8e13b43bfba61c1ee239233515f
Sent transaction 7/700 with hash: 0x49f13e70dab8b59e19a8d07fd29641d0ce3a892c99b72bf7583e4fc29f593325
Sent transaction 8/700 with hash: 0xe00350b34b513e09e52efdbcbe8bf2936dcba07b37fe356ec5733043db469103
Sent transaction 9/700 with hash: 0xdc65fd9d0fc42a4cf4f1e4c93db7057a190df039cd12784ee40292f8eb30c770
Sent transaction 10/700 with hash: 0x8b353815aea015a8b3124988f5472c293a390234bc7ad2a58b48ab

In [16]:
start_time8 = time.time()

generate_transactions(web3, contract, web3.eth.defaultAccount, 80) # send 2000 transactions

end_time8 = time.time()

latency8 = end_time8 - start_time8
tps8 = 8000 / int(latency8)
print(f'Total transactions sent: 8000')
print(f'Latency: {int(latency8)}')
print(f'Transaction per second (tps): {tps8}')

Sent transaction 1/800 with hash: 0xb10c2b9033f63dc9fb66f25eece03280c81414976b5fccbcbab230374170a3e4
Sent transaction 2/800 with hash: 0x4b1e1ad6597a9efa5442148f985392526a6556e8c3a699c5e276e97c91610e15
Sent transaction 3/800 with hash: 0x8033308ed0c12d870e39ecb50b833927661a4e085a494ae4a5cd0cb87e9d632a
Sent transaction 4/800 with hash: 0xdece0907d665abcd38adcf045d91ff0cce9782bf1ee710af53b4fe2c911d293e
Sent transaction 5/800 with hash: 0xcce697795fb4d7036a8f1b700056a6503d43fd0a80663e24a4f10c4bfa2eec9a
Sent transaction 6/800 with hash: 0x7766be485e6b8e7e3f76e25e1c4b36f7b0f19bb7ee7c0537c5650d3765a16eeb
Sent transaction 7/800 with hash: 0x9c13c517f8daedae93dccd2524425e6797bb1ae1203b85b29a4f8674c1e167e1
Sent transaction 8/800 with hash: 0xc674056178a43bad0272cf111a60912b38c2ef74ccc67019b6e347717f588d03
Sent transaction 9/800 with hash: 0x910d6930cbbd66dc618b21f9b046efacf90553aa18a085fa2c38db2e7340218d
Sent transaction 10/800 with hash: 0x3899e1de8abdfd3c9f316787a7b0d16479d5a083fa4d714d0c5b1d

In [17]:
start_time9 = time.time()

generate_transactions(web3, contract, web3.eth.defaultAccount, 90) # send 2000 transactions

end_time9 = time.time()

latency9 = end_time9 - start_time9
tps9 = 9000 / int(latency9)
print(f'Total transactions sent: 9000')
print(f'Latency: {int(latency9)}')
print(f'Transaction per second (tps): {tps9}')

Sent transaction 1/900 with hash: 0x9b6be76d961d7ac6b7ae2c04602f44685be74a861d3b49b7b392f5e081deac90
Sent transaction 2/900 with hash: 0xfd4643d3131c598bdf5ba8a30fc87876cbc5f78d3730a2cac57c9535e3a1c67a
Sent transaction 3/900 with hash: 0xde8e83aa5b5ae384ce09248138465ef91c107457d62fb1491c1983eff4e7a15c
Sent transaction 4/900 with hash: 0x545ee1c11aff965dbc34270adcd7a543b26b06974721e1bbf467f663b0862d15
Sent transaction 5/900 with hash: 0x15655806732fe63c15f645a5afefdb51a6cfeda8ef1c705e6daeeb5e9ca5050c
Sent transaction 6/900 with hash: 0xd8c4d582673d0a8071d22019548792899c4e26ceaa8a8e536ba630dec5344af4
Sent transaction 7/900 with hash: 0x624a52a15ad5f045721b7da7fa55d65a49d4f96e644b92ec62c04e7daf2d5ce7
Sent transaction 8/900 with hash: 0x6f5c94240a5e7266eb7c8b066fb86aa58e9fe073624b54b741a59b4ea7d297bb
Sent transaction 9/900 with hash: 0xf96f881eee8cf851fdd810e8b235f9d9a6d1f751a2a66a5568cbe8cf18f764a6
Sent transaction 10/900 with hash: 0x515b2e79b448c01798522e3cdc805c673799f310ce5ef8ef936bd2

In [18]:
start_time10 = time.time()

generate_transactions(web3, contract, web3.eth.defaultAccount, 100) # send 10000 transactions

end_time10 = time.time()

latency10 = end_time10 - start_time10
tps10 = 10000 / int(latency10)
print(f'Total transactions sent: 10000')
print(f'Latency: {int(latency10)}')
print(f'Transaction per second (tps): {tps10}')

Sent transaction 1/1000 with hash: 0xf8da78e05211df693a93481ebf9cc1f95d0586fa65dd651933826867dfd313b3
Sent transaction 2/1000 with hash: 0xaf8fe19a521a15aded9c5e6da5a087d5d616e8d0269897b7ac6dec433c5cd7b0
Sent transaction 3/1000 with hash: 0x7a1c7889fdc05b8dfde0ed9ab50fb663ee417a51001e15004ad905bb13f6f6a4
Sent transaction 4/1000 with hash: 0x7dd9b5b1d846313f4358eafd8869d712791b92e7a110dda5ce96a89a07b31f16
Sent transaction 5/1000 with hash: 0x3e17b27791c0e3f2020b23688c727c2b2b6f7753a6879715793b7f972011990f
Sent transaction 6/1000 with hash: 0x3f3be39806d7160266b915b5edaf3d7801163d6bb8b82f37e0c813b289ad8bf5
Sent transaction 7/1000 with hash: 0x5630906ada01b6aa1f7cac629ff5f57c3baad408201d25f7d147a8296a25eddb
Sent transaction 8/1000 with hash: 0xa6ab82f19e84c0e4017476ababf3c52a65d6c18a916f6c5ac4bd656f364d8faa
Sent transaction 9/1000 with hash: 0x11e7effae4895a553c6707e8314199d873d9cddf30eda05299d94cb29e4f749f
Sent transaction 10/1000 with hash: 0xcd71911898f6ae03cc2a44b6ffa72e3e4b1d3d66f59f